In this notebook, we will download a model, dataset, and metric from Hugigng Face Hub and generate a interactive HTML Model Card using Intel AI Safety Model Card Generator Tool.

## 1. Download and Import Dependencies

In [ ]:
!pip install evaluate datasets transformers[torch] scikit-learn

In [ ]:
from intel_ai_safety.model_card_gen.model_card_gen import ModelCardGen
from datasets import load_dataset, load_metric
import evaluate
from transformers import AutoConfig,AutoModelForSequenceClassification,AutoTokenizer
import pandas as pd

from collections import Counter
from functools import reduce
import json
import numpy as np

## 2. Download Dataset from Hugging Face Datasets

In [ ]:
raw_dataset = load_dataset("hatexplain")
he_dataset = raw_dataset.map(lambda e: {'text': " ".join(e['post_tokens'])})

## 3. Transform Dataset

In [ ]:
def get_common_targets(elm, ignore=['Other', 'None']):
    """
    This function merges annotated targets from each annotator
    into a single list when annotators agree
    """
    targets = elm['annotators']['target']
    counts = reduce(lambda x, y: Counter(x) + Counter(y) , targets)
    result = [target for target, count in counts.items() if count > 1]
    if result:
        return {'target': result}
    else:
        return {'target': []}

he_dataset = he_dataset.map(get_common_targets)

In [ ]:
def get_top_communites(targets, top=10):
    target_counts = reduce(lambda x, y: Counter(x) + Counter(y) , targets)
    top_targets, _ =  zip(*target_counts.most_common(top))
    return set(top_targets)

TOP = get_top_communites(he_dataset['test']['target'])

def filter_top_target(elm):
    """
    This function filteras the identity groups targeted
    in each item with the top 10 most common identity groups
    """
    targets = set(elm['target']) & TOP
    return {'target': targets}
    
he_dataset = he_dataset.map(filter_top_target)

In [ ]:
def get_label(elm):
    """
    This fuction gets a ground truth label from annotators labels
    """
    label_map = {0: 1, # hatespech -> 1
                 1: 0, # normal -> 0
                 2: 1} # abusive -> 1

    labels = elm['annotators']['label']
    max_label = max(labels, key=labels.count)
    return {'label': label_map[max_label]}
    
he_dataset = he_dataset.map(get_label)

## 4. Download Modle and Process Outputs

In [ ]:
from torch.nn.functional import softmax

he_dataset.set_format("pt", columns=["post_tokens"], output_all_columns=True)
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")

def process(examples):
    bert_tokens =  tokenizer(examples['text'], return_tensors="pt")
    output = model(**bert_tokens)
    return {"output": softmax(output['logits'], dim=-1).flatten()}

test_ds = he_dataset['test'].map(process)

## 5. Get Bias Metric form Hugging Face

In [ ]:
metric = evaluate.load('Intel/bias_auc')
print(metric)

## 6. Run Bias Metric

In [ ]:
metric.add_batch(target=test_ds['target'],
                 label=test_ds['label'],
                 output=test_ds['output'])

subgroups = set(group for group_list in test_ds['target'] for group in group_list) - set(['Disability'])

metric_output = metric.compute(subgroups = subgroups)

In [ ]:
metric_output

## 7. Transform Output for Model Card

Mode Card Generator take two pandas dataframes as input. We will creat a `metrics_by_group` dataframe from the Bias AUC metric above as well as a `metrics_by_threshold` containing performance metrics at threshold.

In [ ]:
metrics_by_group = (pd.DataFrame.from_dict(metric_output).
      T.
      reset_index().
      rename({'index': 'group'}, axis=1))
metrics_by_group['feature'] = ['target'] * len(metrics_by_group)
metrics_by_group

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
thetas = np.linspace(0, 1, 1001)
y_pred_prob = test_ds['output'][:,1]

metrics_dict ={
    'threshold': thetas,
    'precision': [precision_score(test_ds['label'], y_pred_prob > theta) for theta in thetas],
    'recall': [recall_score(test_ds['label'], y_pred_prob > theta) for theta in thetas],
    'f1': [f1_score(test_ds['label'], y_pred_prob > theta) for theta in thetas],
    'accuracy' : [accuracy_score(test_ds['label'], y_pred_prob > theta) for theta in thetas]
}

In [ ]:
metrics_by_threshold = pd.DataFrame.from_dict(metrics_dict)

In [ ]:
metrics_by_threshold

## 8. Build Model Card

Simply add the dataframes into the `ModelCardGen.generate` class method to build a model card.

In [ ]:
mc =  {
    "schema_version": "0.0.1",
    "model_details": {
        "name": "Deep Learning Models for Multilingual Hate Speech Detection",
        "version": {
            "name": "25d0e4d9122d2a5c283e07405a325e3dfd4a73b3",
            "date": "2020"
        },
        "graphics": {},

        "citations": [
             {
                "citation": '''@article{aluru2020deep,
                title={Deep Learning Models for Multilingual Hate Speech Detection},
                author={Aluru, Sai Saket and Mathew, Binny and Saha, Punyajoy and Mukherjee, Animesh},
                journal={arXiv preprint arXiv:2004.06465},
                year={2020}
                }'''
             },
        ],
        "overview": 'This model is used detecting hatespeech in English language. The mono in the name refers to the monolingual setting, where the model is trained using only English language data. It is finetuned on multilingual bert model. The model is trained with different learning rates and the best validation score achieved is 0.726030 for a learning rate of 2e-5. Training code can be found here https://github.com/punyajoy/DE-LIMIT',
    }
}

In [ ]:
mcg = ModelCardGen.generate(metrics_by_group=metrics_by_group, metrics_by_threshold=metrics_by_threshold, model_card=mc)
mcg

In [ ]:
mcg.export_html('ModelCard.html')

In [ ]:
from IPython.display import display, HTML
from plotly.io import to_html, templates
from plotly.offline import init_notebook_mode, get_plotlyjs
fig = px.line(x=["a","b","c"], y=[1,3,2], title="sample figure")
html_content = to_html(fig, include_plotlyjs="require", full_html=False)


In [ ]:
HTML(html_content)